# Interactions

In [6]:
import sys
import glob
import numpy as np
import pandas as pd
from collections import defaultdict

sys.path.append('/home/kalkiek/projects/reddit-political-affiliation/')

from src.data.date_helper import read_submissions
from src.features.interactions.political_comment import PoliticalComment

### Quick Clean Up

I forgot to  remove 't1' prefix from some of the comment IDs

We also need to remove any bot accounts

In [2]:
def grab_bot_accounts():
    fname = '/shared/0/projects/prosocial/known-bots.tsv'
    bots = []

    with open(fname, 'rt') as f:
        lines = f.readlines()

        for line in lines:
            bots.append(line.split('\t')[1])

    print("Known bots: %d" % len(bots))
    return bots

bots = grab_bot_accounts()
bots.extend('[deleted]')

Known bots: 393


###  Can't run this until the script is done running! We'll run it in a few days

In [6]:
def remove_id_prefix_and_bots(in_file):
    comments = 0
    with open(in_file, 'r') as f:
        for line in f:
            comment_id, parent_id, username, subreddit, created, politics, text = line.split('\t')
            if username not in bots:
                if comment_id[:2] == 't1':
                    comment_id = comment_id[3:]
                if parent_id[:2] == 't1':
                    parent_id = parent_id[3:]

                comments += 1
                political_comment = PoliticalComment(comment_id, parent_id, username, subreddit, created, politics, text)
                yield political_comment

    print("Total number of political comments: {}".format(comments))

in_file = '/shared/0/projects/reddit-political-affiliation/data/interactions/all_comments.tsv'
out_file = '/shared/0/projects/reddit-political-affiliation/data/interactions/all_comments_filtered.tsv'

with open(out_file, 'w') as out_f:
    for comment in filtered_comments(in_file):
        out_f.write(comment.to_tsv_row())

Total number of political comments: 113921968


## Reading In The Data

Sample of how to read the data in

In [9]:
in_file = '/shared/0/projects/reddit-political-affiliation/data/interactions/comment_ids.tsv'

def read_in_comments(in_file, count=-1):
    comments = []
    with open(in_file, 'r') as f:
        for line in f:
            line = line.strip()
            comment_id, parent_id, username, subreddit, created, politics = line.split('\t')
            political_comment = PoliticalComment(comment_id, parent_id, username, subreddit, created, politics)
            comments.append(political_comment.to_dict())
            if count > 0 and len(comments) >= count:
                return comments
            
    print("Total number of political comments: {}".format(len(comments)))
    return comments


        
comments = read_in_comments(in_file, count=100)
df_comments = pd.DataFrame(comments)
df_comments.head()

,comment_id,parent_id,username,subreddit,created,politics,text
0,eypc9zv,t1_eyp9duh,CueDramaticMusic,tumblr,1567296002,Democrat,
1,eypca5l,t1_eypby8b,ninethirtyone,news,1567296005,Democrat,
2,eypca5t,t3_cy2nlj,lostapwbm,SquaredCircle,1567296006,Democrat,
3,eypcabv,t1_eyonlbn,needler14,ABoringDystopia,1567296009,Democrat,
4,eypcac1,t3_cy09os,LimpWibbler_,funny,1567296009,Democrat,


# Bucket about dyadic affiliation composition (e.g,. Liberal+Liberal)

# Comments in same subreddit made at a similar time point